In [22]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from urllib.parse import urljoin

# Function to extract Product Title
def get_title(soup):
    try:
        title = soup.find("span", attrs={"id": 'productTitle'})
        title_value = title.text.strip()
    except AttributeError:
        title_value = ""
    return title_value

# Function to extract Product Price
def get_price(soup):
    try:
        price = soup.find("span", attrs={'class': 'a-offscreen'}).string.strip()
    except AttributeError:
        price = ""
    return price

# Function to extract Product Rating
def get_rating(soup):
    try:
        rating = soup.find("span", attrs={'class': 'a-icon-alt'}).string.strip()
    except AttributeError:
        rating = ""
    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id': 'acrCustomerReviewText'}).string.strip()
    except AttributeError:
        review_count = ""
    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id': 'availability'})
        available = available.find("span").string.strip()
    except AttributeError:
        available = "Not Available"
    return available

if __name__ == '__main__':
    # Add your user agent (replace '' with an actual user agent string)
    HEADERS = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36',
        'Accept-Language': 'en-US, en;q=0.5'
    }

    # The webpage URL
    URL = "https://www.amazon.com/s?k=playstation+4&ref=nb_sb_noss_2"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Check if the request was successful
    if webpage.status_code != 200:
        print(f"Failed to fetch the page. Status code: {webpage.status_code}")
    else:
        # Soup Object containing all data
        soup = BeautifulSoup(webpage.content, "html.parser")

        # Fetch links as List of Tag Objects
        links = soup.find_all("a", attrs={'class': 'a-link-normal s-no-outline'})

        # Store the links
        links_list = []

        # Loop for extracting links from Tag Objects
        for link in links:
            # Use urljoin to handle relative URLs
            full_link = urljoin("https://www.amazon.com", link.get('href'))
            links_list.append(full_link)

        # Dictionary to store product details
        d = {"title": [], "price": [], "rating": [], "reviews": [], "availability": []}

        # Loop for extracting product details from each link
        for link in links_list:
            try:
                new_webpage = requests.get(link, headers=HEADERS)
                new_soup = BeautifulSoup(new_webpage.content, "html.parser")

                # Function calls to extract product information
                d['title'].append(get_title(new_soup))
                d['price'].append(get_price(new_soup))
                d['rating'].append(get_rating(new_soup))
                d['reviews'].append(get_review_count(new_soup))
                d['availability'].append(get_availability(new_soup))
            except Exception as e:
                print(f"Error processing {link}: {e}")

        # Create a DataFrame from the collected data
        amazon_df = pd.DataFrame.from_dict(d)

        # Replace empty titles with NaN and drop rows with missing titles
        amazon_df['title'] = amazon_df['title'].replace('', np.nan)
        amazon_df = amazon_df.dropna(subset=['title'])

        # Replace empty values with "Not Available" for other columns
        amazon_df.fillna({
            "price": "Not Available",
            "rating": "Not Available",
            "reviews": "Not Available",
            "availability": "Not Available"
        }, inplace=True)

        # Save the data to a CSV file
        amazon_df.to_csv("amazon_data.csv", header=True, index=False)

        # Display the DataFrame
        print(amazon_df)

Error processing https://www.amazon.com/Sony-PlayStation-Premium-Bundle-Renewed-4/dp/B0DLHQ94B4/ref=sr_1_4?dib=eyJ2IjoiMSJ9.DYCLh1lfDEqmNIOXcI70zeBtbVDEVFdpQdh7MnKCAqbRsvzxVzTS22FOKdE-QhogNTEMxVo_haqWHA8Puo7aKxTN_rozLsrXWh0qPZfvGn9b5XlWqCE5uO2lfbAYF809hho_KiegrvR8NZ_m5G5S15qSfB_UNs69QeWBl9g74QitjdEsuRqT1xSQiWGL5iHXOl5z3CbgBA_uzx-IyXr_zd_cO0u1hlhz-23A_tyG4yc.YtgpcDED-4F238Fb5UhBK9inbNyS4HVb8w-pV3V8ZfU&dib_tag=se&keywords=playstation+4&qid=1738532387&sr=8-4: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
                                                title    price  \
0                      PlayStation 4 Slim 1TB Console  $369.95   
1              Sony Playstation PS4 1TB Black Console  $200.00   
2   Playstation Sony 4, 500GB Slim System [CUH-221...  $189.99   
3   PlayStation®4 Console – Call of Duty® Modern W...  $379.92   
4                      PlayStation 4 Slim 1TB Console  $249.99   
5   Sony PlayStation 4 Pro 1TB Console - Black (PS